In [9]:
import pandas as pd

# Load the CSV file with question, answer key, and marks
file_path = r"data\questionAndAnswers.txt"
df = pd.read_csv(file_path,delimiter="|")

# Example of how your DataFrame looks
print(df)

  Qno                      Question Description  \
0  Q1            Which is the Capital of India?   
1  Q2  What are Geographical Benifits of India?   

                                              Answer  Max Marks  
0                                          New Delhi          1  
1  India’s Strategic Location.Isolation and Conne...          2  


In [10]:
from sentence_transformers import SentenceTransformer

# Initialize the HuggingFace Sentence Transformer model (or any other HuggingFace model)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare the text for embedding
questions = df['Question Description'].tolist()
correct_answers = df['Answer'].tolist()

# Example student answers (can be stored in a separate list or loaded)
student_answers = [
    "Delhi",  # Student's answer for Q1
    "India’s Strategic Location.Isolation and Connectivity"  # Student's answer for Q2
]

# Create embeddings for questions, correct answers, and student answers
question_embeddings = model.encode(questions)
correct_answer_embeddings = model.encode(correct_answers)
student_answer_embeddings = model.encode(student_answers)

# Print the embeddings (optional, as the embeddings are large vectors)
print(question_embeddings[0])  # Embedding for the first question

d:\LangChain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[ 9.84213203e-02  1.79742165e-02 -4.64241616e-02  3.91058400e-02
 -2.82076895e-02 -5.09275384e-02  5.81051409e-02  1.74796041e-02
 -1.69745795e-02 -2.38932371e-02 -1.66553464e-02 -1.35838196e-01
  3.39705460e-02 -3.29496339e-02  1.92225948e-02 -8.85911509e-02
  2.90319435e-02 -8.57441593e-03  9.75931734e-02 -6.88676089e-02
  7.35698268e-05  2.75427382e-02 -3.27298837e-03 -7.87428916e-02
  6.98106885e-02  2.27475967e-02  3.96960750e-02 -4.10304479e-02
 -1.39524927e-02 -7.88887683e-03  5.07853068e-02 -5.13138510e-02
 -7.52076413e-03  4.53391159e-03 -6.01102710e-02 -3.44084087e-03
 -3.95009145e-02  5.85709512e-02  1.60099700e-01 -5.28912768e-02
  1.80117041e-02  1.75884739e-02  6.89322874e-02  1.33438278e-02
  4.67530861e-02 -1.62491724e-02  4.91726492e-03  1.77645907e-02
  1.30894678e-02 -6.36175424e-02  7.42257610e-02 -4.75886539e-02
 -7.92552456e-02  2.21456066e-02 -1.05005121e-02 -3.66385072e-03
 -3.40875797e-02 -4.48063053e-02 -1.59561243e-02  5.95230125e-02
 -2.75363363e-02  8.67148

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between student answer and correct answer
similarities = []
for correct_embedding, student_embedding in zip(correct_answer_embeddings, student_answer_embeddings):
    similarity = cosine_similarity([correct_embedding], [student_embedding])
    similarities.append(similarity[0][0])

print(similarities)

[0.86936057, 0.83812904]


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
os.environ["OPENAI_API_KEYKEN"]=os.getenv("OPENAI_API_KEY")

In [ ]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def generate_score(question, correct_answer, student_answer, max_marks):
    # Create the prompt for GPT-4
    prompt = f"""
    Question: {question}
    Correct Answer: {correct_answer}
    Student's Answer: {student_answer}
    Max Marks: {max_marks}
    Based on the student's answer, assign a score out of {max_marks} by evaluating how similar the student's answer is to the correct answer. Provide a detailed explanation of the score.
    """

    # Call GPT-4 API to evaluate the answer
    response = openai.Completion.create(
        model="gpt-4",
        prompt=prompt,
        max_tokens=100
    )

    # Extract the score from the response
    return response.choices[0].text.strip()

# Example of evaluating the first question
question = df.loc[0, 'Question Description']
correct_answer = df.loc[0, 'Answer']
student_answer = student_answers[0]
max_marks = df.loc[0, 'Max Marks']

# Generate the score using GPT-4
score = generate_score(question, correct_answer, student_answer, max_marks)
print(score)